<a href="https://colab.research.google.com/github/thainhf/proj-me-db-data/blob/main/Clean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q gspread
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
! pip install urllib2
! pip install urllib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 22.0 MB 55 kB/s
     |████████████████████████████████| 102 kB 5.1 MB/s 
     |████████████████████████████████| 690 kB 40.2 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 9.8 MB 37.3 MB/s 
     |████████████████████████████████| 4.7 MB 32.2 MB/s 
     |████████████████████████████████| 296 kB 47.0 MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.4.0-py2.py3-none-any.whl size=315269 sha256=f40c90f97a4fb437e0ceaa63b00208ba8cb4c168af118ac440a31ea085fda7ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-urlltpk9/wheels/cc/d5/09/083fb07c9363a2f45854b0e3a7de7d7c560f07da74b9e9769d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=fdbd1fec72e7d4f8215bf7610166ebc67765b6fcbec8089997b9b67697d2a090
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c

In [ ]:
import gspread
import pandas as pd
import numpy as np
from google.colab import auth
auth.authenticate_user()
from google.auth import default
from pathlib import Path  
import re
from pandas_profiling import ProfileReport

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/(ThaiNHF) ข้อมูลพัฒนาเสนอโครงการ สสส/Copy of 650411_ฐานข้อมูลเครือข่ายนักประเมิน_จากลิ้ง.xlsx', index_col=0, header=2)

# แก้ option ตอน read ให้เป็น utf-8

In [ ]:
df.columns

Index(['คำนำหน้า', 'ตำแหน่ง', 'ชื่อ', 'นามสกุล', 'ชื่อ-นามสกุล', 'ที่อยู่',
       'โทรศัพท์', 'โทรศัพท์มือถือ', 'อีเมล์', 'ด้านการศึกษา', 'Unnamed: 11',
       'Unnamed: 12', 'หน่วยงานที่สังกัด', 'ความสนใจด้านการประเมินผล',
       'การทำงานด้านการประเมินผล', 'ประสบการณ์ด้านการประเมินผล',
       'ระดับประสบการณ์ด้านการประเมินผล',
       'การทำงานด้านประเมินผลงานสร้างเสริมสุขภาพ',
       'กรอบการประเมินผล (Evaluation frameworks) ที่ใช้ประเมินผล',
       'รูปแบบดำเนินการ (Evaluation approaches) ที่ใช้ประเมินผล',
       'ความเข้าใจต่อภารกิจของ สสส.', 'การประเมินโครงการ สสส.',
       'ฐานะผู้ประเมิน', 'เคยประเมินผลให้กับหน่วยงานอื่น', 'ความเชี่ยวชาญ',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'ความเชี่ยวชาญด้านการวิเคราะห์ข้อมูล', 'Unnamed: 30', 'Unnamed: 31',
       'Unnamed: 32', 'ปัญหาและอุปสรรคจากการประเมินฯ',
       'ข้อเสนอแนะต่อการประเมินฯ', 'ข้อเสนอแนะต่อการพัฒนาศักยภาพนักประเมินฯ',
       'ฐาน สสส.', 'รวมโดย M&E', 'อบรมนักประเมิน', 'DE', 'เครือข่ายหมอพงค์เทพ',
   

## **1. Changing columns name**

In [ ]:
recol_df = df.copy()
n=0
for key in df.columns:
  if not recol_df.iloc[:1,n:n+1].isnull().values.any() :
    #print(df.columns[n], '->', df.iloc[0,n])
    if 'Unnamed' not in recol_df.columns[n] :
      print(recol_df.columns[n], '->', recol_df.columns[n]+ ' ' +recol_df.iloc[0,n])
      recol_df = recol_df.rename(columns={recol_df.columns[n]:recol_df.columns[n]+ ' ' +recol_df.iloc[0,n]})

    else:
      a=n
      while 'Unnamed' in recol_df.columns[a]:
        a -= 1
      else:
        print(recol_df.columns[n], '->', recol_df.columns[a].split()[0]+ ' ' +recol_df.iloc[0,n])
        recol_df = recol_df.rename(columns={recol_df.columns[n]:recol_df.columns[a].split()[0]+ ' ' +recol_df.iloc[0,n]})
        
    #new_df = df.rename(columns={df.columns[n]:df.iloc[0,n]})
  n += 1


ด้านการศึกษา -> ด้านการศึกษา ระดับการศึกษา
Unnamed: 11 -> ด้านการศึกษา สาขาที่จบ (สูงสุด)
Unnamed: 12 -> ด้านการศึกษา สถาบันการศึกษาที่จบ (สูงสุด)
ความเชี่ยวชาญ -> ความเชี่ยวชาญ ด้านประเด็น
Unnamed: 26 -> ความเชี่ยวชาญ กลุ่มเป้าหมาย
Unnamed: 27 -> ความเชี่ยวชาญ พื้นที่ดำเนินการ
Unnamed: 28 -> ความเชี่ยวชาญ ทักษะ skill and competency
ความเชี่ยวชาญด้านการวิเคราะห์ข้อมูล -> ความเชี่ยวชาญด้านการวิเคราะห์ข้อมูล เชิงคุณภาพ
Unnamed: 30 -> ความเชี่ยวชาญด้านการวิเคราะห์ข้อมูล เชิงปริมาณแบบ descriptive study
Unnamed: 31 -> ความเชี่ยวชาญด้านการวิเคราะห์ข้อมูล เชิงปริมาณแบบ analytical study
Unnamed: 32 -> ความเชี่ยวชาญด้านการวิเคราะห์ข้อมูล เชิงปริมาณแบบ experimental study


In [ ]:
recol_df.columns

Index(['คำนำหน้า', 'ตำแหน่ง', 'ชื่อ', 'นามสกุล', 'ชื่อ-นามสกุล', 'ที่อยู่',
       'โทรศัพท์', 'โทรศัพท์มือถือ', 'อีเมล์', 'ด้านการศึกษา ระดับการศึกษา',
       'ด้านการศึกษา สาขาที่จบ (สูงสุด)',
       'ด้านการศึกษา สถาบันการศึกษาที่จบ (สูงสุด)', 'หน่วยงานที่สังกัด',
       'ความสนใจด้านการประเมินผล', 'การทำงานด้านการประเมินผล',
       'ประสบการณ์ด้านการประเมินผล', 'ระดับประสบการณ์ด้านการประเมินผล',
       'การทำงานด้านประเมินผลงานสร้างเสริมสุขภาพ',
       'กรอบการประเมินผล (Evaluation frameworks) ที่ใช้ประเมินผล',
       'รูปแบบดำเนินการ (Evaluation approaches) ที่ใช้ประเมินผล',
       'ความเข้าใจต่อภารกิจของ สสส.', 'การประเมินโครงการ สสส.',
       'ฐานะผู้ประเมิน', 'เคยประเมินผลให้กับหน่วยงานอื่น',
       'ความเชี่ยวชาญ ด้านประเด็น', 'ความเชี่ยวชาญ กลุ่มเป้าหมาย',
       'ความเชี่ยวชาญ พื้นที่ดำเนินการ',
       'ความเชี่ยวชาญ ทักษะ skill and competency',
       'ความเชี่ยวชาญด้านการวิเคราะห์ข้อมูล เชิงคุณภาพ',
       'ความเชี่ยวชาญด้านการวิเคราะห์ข้อมูล เชิงปริมาณแบบ descriptive stud

In [ ]:
recol_df.columns.get_loc("ปัญหาและอุปสรรคจากการประเมินฯ")

32

## **2. Selecting important columns & row**

In [ ]:
new_df = recol_df.iloc[1:622,:32]
#new_df = new_df.drop(columns=['ชื่อ-นามสกุล', 'ความเข้าใจต่อภารกิจของ สสส.', 'การประเมินโครงการ สสส.', 'ฐานะผู้ประเมิน'], axis=1)
new_df = new_df.drop(columns=['ชื่อ-นามสกุล', 'ความเข้าใจต่อภารกิจของ สสส.', 'การประเมินโครงการ สสส.'], axis=1)
new_df

,คำนำหน้า,ตำแหน่ง,ชื่อ,นามสกุล,ที่อยู่,โทรศัพท์,โทรศัพท์มือถือ,อีเมล์,ด้านการศึกษา ระดับการศึกษา,ด้านการศึกษา สาขาที่จบ (สูงสุด),...,ฐานะผู้ประเมิน,เคยประเมินผลให้กับหน่วยงานอื่น,ความเชี่ยวชาญ ด้านประเด็น,ความเชี่ยวชาญ กลุ่มเป้าหมาย,ความเชี่ยวชาญ พื้นที่ดำเนินการ,ความเชี่ยวชาญ ทักษะ skill and competency,ความเชี่ยวชาญด้านการวิเคราะห์ข้อมูล เชิงคุณภาพ,ความเชี่ยวชาญด้านการวิเคราะห์ข้อมูล เชิงปริมาณแบบ descriptive study,ความเชี่ยวชาญด้านการวิเคราะห์ข้อมูล เชิงปริมาณแบบ analytical study,ความเชี่ยวชาญด้านการวิเคราะห์ข้อมูล เชิงปริมาณแบบ experimental study
1.0,นาง,NaN,กนกพร,ตันวัฒนะ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,น.ส.,NaN,กนกภรณ์,ชูเชิด,เลขที่ 119/36 ซอยท่าอิฐ หมู่บ้านมณียา ถนนรัตนา...,NaN,0899241928,chucherd_k@lycos.com,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,คุณ,NaN,กนกรัตน์,กิตติวิวัฒน์,สาขาพัฒนาสังคม คณะมนุษยศาสตร์และสังคมศาสตร์ มห...,NaN,0816237579,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,ดร.,กนิษฐ์,ศรีเคลือบ,ภาควิชาวิจัยและจิตวิทยาการศึกษา คณะครุศาสตร์ จ...,NaN,0836157722,ksriklaub@gmail.com,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,นาง,รศ.ดร.,กนิษฐา,จำรูญสวัสดิ์,ภาควิชาอนามัยครอบครัว คณะสาธารณสุขศาสตร์ มหาวิ...,02-3548536,0816511353,kanittha.cha@mahidol.ac.th,ปริญญาเอก,ปร.ด. (สังคมศาสตร์การแพทย์),...,ผู้ประเมินภายนอก,1. สำนักอนามัยการเจริญพันธุ์ กระทรวงสาธารณสุข\...,"กลุ่มโรคไม่ติดต่อ, ระบบสุขภาพ, สุขภาวะทางเพศ","กลุ่มเด็กวัยรุ่น/นักศึกษา(15-21ปี), กลุ่มวัยทำ...",NaN,NaN,มาก,มากที่สุด,มากที่สุด,มากที่สุด
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617.0,นาง,รศ.ดร.,อุษา,บิ้กกิ้นส์,5/231 ม.ประชานิเวศน์ 2ระยะ 3 ซ. สามัคคี 65 ถ.ส...,0869792160,0869792160,O_SUKSAI@hotmail.com,ปริญญาเอก,Ph.D. ( Communication),...,ผู้ประเมินภายนอก,Thai PBS,"ระบบสุขภาพ, สุขภาพจิต, อุบัติเหตุ, รู้เท่าทันส...","เด็กปฐมวัย (0-5ปี), เด็กวัยเรียน (5-14ปี), กลุ...",NaN,NaN,มากที่สุด,มาก,มาก,มาก
618.0,คุณ,NaN,อุษา,อติโภคบูรณ์,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
619.0,NaN,ผศ.ดร.,อุษา,อ้นทอง,เลขที่ 140 ถนนกาญจนวานิช ต.เขารูปช้าง อ.เมือง ...,NaN,"0869614556, 0819654556",usaonthong@hotmail.com,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
620.0,น.ส.,ดร.,อุษารดี,ภู่มาลี,คณะวนศาสตร์ มหาวิทยาลัยเกษตรศาสตร์ 50 ถนนงามวง...,0930238420,0930238420,ploy_ku59@hotmail.com,ปริญญาเอก,ปร.ด (อุทยาน นันทนาการ และการท่องเที่ยว),...,"ผู้ประเมินภายใน, ผู้ประเมินภายนอก",กรมส่งเสริมคุณภาพสิ่งแวดล้อม กรมทรัพยากรทะเลแล...,"สิ่งแวดล้อม, การพัฒนาชุมชน","กลุ่มเด็กวัยรุ่น/นักศึกษา(15-21ปี), กลุ่มวัยทำ...",NaN,NaN,มาก,มากที่สุด,มากที่สุด,มาก


## **3. Split data**

In [ ]:
new_df.isnull().sum()

คำนำหน้า                                                                191
ตำแหน่ง                                                                 312
ชื่อ                                                                      0
นามสกุล                                                                   0
ที่อยู่                                                                 206
โทรศัพท์                                                                462
โทรศัพท์มือถือ                                                          208
อีเมล์                                                                  158
ด้านการศึกษา ระดับการศึกษา                                              417
ด้านการศึกษา สาขาที่จบ (สูงสุด)                                         423
ด้านการศึกษา สถาบันการศึกษาที่จบ (สูงสุด)                               423
หน่วยงานที่สังกัด                                                       589
ความสนใจด้านการประเมินผล                                                423
การทำงานด้าน

In [ ]:
"""profile = ProfileReport(new_df, minimal=True,explorative=True)
# for large dataset minimal is True, and explorative as True
profile.widgets()"""

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 3609 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/usr/local/lib/python3.7/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 3626 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/usr/local/lib/python3.7/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 3634 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/usr/local/lib/python3.7/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 3618 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/usr/local/lib/python3.7/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 3588 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/usr/local/lib/python3.7/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 3640 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
new_df['กรอบการประเมินผล (Evaluation frameworks) ที่ใช้ประเมินผล'] = new_df['กรอบการประเมินผล (Evaluation frameworks) ที่ใช้ประเมินผล'].str.split(pat="( , )|(, )|( ,)")
new_df['รูปแบบดำเนินการ (Evaluation approaches) ที่ใช้ประเมินผล'] = new_df['รูปแบบดำเนินการ (Evaluation approaches) ที่ใช้ประเมินผล'].str.split(pat="( , )|(, )|( ,)")
#new_df['รูปแบบดำเนินการ (Evaluation approaches) ที่ใช้ประเมินผล'] = new_df['รูปแบบดำเนินการ (Evaluation approaches) ที่ใช้ประเมินผล'].str.split(pat="( , )|(, )|( ,)")
new_df['ความเชี่ยวชาญ ด้านประเด็น'] = new_df['ความเชี่ยวชาญ ด้านประเด็น'].str.split(pat="( , )|(, )|( ,)")
new_df['ความเชี่ยวชาญ กลุ่มเป้าหมาย'] = new_df['ความเชี่ยวชาญ กลุ่มเป้าหมาย'].str.split(pat="( , )|(, )|( ,)")
new_df['ความเชี่ยวชาญ พื้นที่ดำเนินการ'] = new_df['ความเชี่ยวชาญ พื้นที่ดำเนินการ'].str.split(pat="( , )|(, )|( ,)")
new_df['ระดับประสบการณ์ด้านการประเมินผล'] = new_df['ระดับประสบการณ์ด้านการประเมินผล'].str.split(pat="( , )|(, )|( ,)")
new_df['เคยประเมินผลให้กับหน่วยงานอื่น'] = new_df['เคยประเมินผลให้กับหน่วยงานอื่น'].str.split(pat="( , )|(, )|( ,)")
#new_df['เคยประเมินผลให้กับหน่วยงานอื่น'] = new_df['เคยประเมินผลให้กับหน่วยงานอื่น'].str.split(pat="[\d\.]+")
new_df['อีเมล์'] = new_df['อีเมล์'].str.split(pat="( , )|(, )|( ,)")

new_df['เคยประเมินผลให้กับหน่วยงานอื่น'][5]

['1. สำนักอนามัยการเจริญพันธุ์ กระทรวงสาธารณสุข\n2. กองควบคุมโรคเอดส์ กทม.\n3. สำนักโรคไม่ติดต่อ กระทรวงสาธารณสุข']

In [ ]:
new_df['ความเชี่ยวชาญ ด้านประเด็น'][5]

['กลุ่มโรคไม่ติดต่อ',
 None,
 ', ',
 None,
 'ระบบสุขภาพ',
 None,
 ', ',
 None,
 'สุขภาวะทางเพศ']

In [ ]:
id = []
key = 999
for i in range(len(new_df)):
  key = key+1
  id.append(key)

print(id)
new_df.insert(loc=0, column='ID', value=id)

[1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 116

### Split Tables

In [ ]:
member_df = new_df.iloc[:,:15]
member_df.sample(5)

,ID,คำนำหน้า,ตำแหน่ง,ชื่อ,นามสกุล,ที่อยู่,โทรศัพท์,โทรศัพท์มือถือ,อีเมล์,ด้านการศึกษา ระดับการศึกษา,ด้านการศึกษา สาขาที่จบ (สูงสุด),ด้านการศึกษา สถาบันการศึกษาที่จบ (สูงสุด),หน่วยงานที่สังกัด,ความสนใจด้านการประเมินผล,การทำงานด้านการประเมินผล
152.0,1151,น.ส.,อ.ดร.,ธัญภรณ์,เกิดน้อย,21 นิมมานเหมินท์ ซอย 17 ต.สุเทพ อ.เมือง เชียงใ...,NaN,0816738960,[tkniman@gmail.com],ปริญญาเอก,การใช้ที่ดินและการจัดการทรัพยากรธรรมชาติอย่างย...,มหาวิทยาลัยเชียงใหม่,NaN,สนใจ,เคย
420.0,1419,NaN,ดร.,วิภาพร,โชคอวยชัย,วิทยาลัยพยาบาลกองทัพเรือ กรมแพทย์ทหารเรือ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266.0,1265,นาย,NaN,พงษ์สุวรรณ,ศรีสุวรรณ,131/2 แยก 42 ซอยพหลโยธิน52 ถนนพหลโยธิน แขวงคลอ...,02-5513675,0909706691,[gp.capt.dr.tor@gmail.com],ปริญญาเอก,วิธีวิทยาการวิจัยการศึกษา,จุฬาลงกรณ์มหาวิทยาลัย,NaN,สนใจ,เคย
69.0,1068,คุณ,NaN,จุมพล,ปริตรโตทก,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97.0,1096,นาย,NaN,ชาตรี,เกื้อศิริกุล,เลขที่ 98/143 ซ กุศลศิลป์ บางนา กทม 10260,NaN,NaN,[Chatree01@hotmail.com],NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def stack_dat(dat):
  dat = dat.explode(dat.columns[1])
  check = dat[dat.columns[1]]
  dat = dat[(check.str.contains(',|None|\([\w+\)]', regex=True) == False) & (check != '') & (check != '-') & (check != 'etc') & (check != 'etc.') & (check != 'ฯลฯ') & (check != '/')]
  #dat[dat.columns[1]] = dat[dat.columns[1]].str.replace(r'^ +', '', regex=True)
  dat[dat.columns[1]] = dat[dat.columns[1]].str.strip()
  dat = dat.dropna().reset_index(drop=True)
  return dat

In [ ]:
# split ตามวรรค ไปก่อน, เอาตัวย่อในวงเล็บออก แล้วเปลี่ยนตัวย่อนอกวงเล็บ เป็นชื่อเต็ม 


evaFrame_df = new_df[['ID', 'กรอบการประเมินผล (Evaluation frameworks) ที่ใช้ประเมินผล']]
evaAppo_df = new_df[['ID', 'รูปแบบดำเนินการ (Evaluation approaches) ที่ใช้ประเมินผล']]
issue_df = new_df[['ID', 'ความเชี่ยวชาญ ด้านประเด็น']]
target_df = new_df[['ID', 'ความเชี่ยวชาญ กลุ่มเป้าหมาย']]
area_df = new_df[['ID', 'ความเชี่ยวชาญ พื้นที่ดำเนินการ']]
level_df = new_df[['ID', 'ระดับประสบการณ์ด้านการประเมินผล']]
unit_df = new_df[['ID', 'เคยประเมินผลให้กับหน่วยงานอื่น']]

evaFrame_df = stack_dat(evaFrame_df)
#issue_df['ความเชี่ยวชาญ ด้านประเด็น'] = issue_df['ความเชี่ยวชาญ ด้านประเด็น'].str.split(pat="/|\\u200b")

evaAppo_df = stack_dat(evaAppo_df)

issue_df = stack_dat(issue_df)
issue_df['ความเชี่ยวชาญ ด้านประเด็น'] = issue_df['ความเชี่ยวชาญ ด้านประเด็น'].str.split(pat="/|\\u200b")
issue_df = stack_dat(issue_df)

target_df = stack_dat(target_df)
target_df['ความเชี่ยวชาญ กลุ่มเป้าหมาย'] = target_df['ความเชี่ยวชาญ กลุ่มเป้าหมาย'].str.split(pat="/|\\u200b")
target_df = stack_dat(target_df)

area_df = stack_dat(area_df)
area_df['ความเชี่ยวชาญ พื้นที่ดำเนินการ'] = area_df['ความเชี่ยวชาญ พื้นที่ดำเนินการ'].str.replace(r' \(วัด/โบสถ์/มัสยิด\)', '')
area_df['ความเชี่ยวชาญ พื้นที่ดำเนินการ'] = area_df['ความเชี่ยวชาญ พื้นที่ดำเนินการ'].str.split(pat="/")
area_df = stack_dat(area_df)

level_df = stack_dat(level_df)

unit_df = stack_dat(unit_df)
unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'] = unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'].str.replace('มหาิท', 'มหาวิท')
unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'] = unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'].str.replace('กระทรง', 'กระทรวง')
unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'] = unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'].str.replace('\u200b', '')
unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'] = unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'].str.replace(' ฯลฯ', '')
unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'] = unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'].str.replace(' เป็นต้น', '')
unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'] = unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'].str.replace('(\d\.)', '')
unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'] = unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'].str.replace('/', '')
unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'] = unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'].str.replace('\(\w+\.\)', '')
unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'] = unit_df['เคยประเมินผลให้กับหน่วยงานอื่น'].str.split(pat=" / |\n ")
unit_df = stack_dat(unit_df)


print(len(set(unit_df[unit_df.columns[1]])))
set(unit_df[unit_df.columns[1]])
#issue_df

183


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: FutureWarning: The default value of regex will change from True to False in a future version.


{'Asian Development Bank',
 'EU',
 'HIVAIDS global fund',
 'ITA',
 'NECTEC',
 'NGO หนึ่ง',
 'Nesdc',
 'PLAN',
 'TIJ',
 'TRIS',
 'Thai PBS',
 'The United Nations Population Fund',
 'UNDP สำนักงาน ปปส. สำนักงาน กปร.',
 'UNFPA',
 'UNICEF',
 'WHOSEARO',
 'World Education',
 'World Health Organization',
 'World vision',
 'กรมการป้องกันและบรรเทาสาธารณภัย',
 'กรมกิจการเด็กและเยาวชน กระทรวงการพัฒนาสังคมและความมั่นคงของมนุษย์',
 'กรมควบคุมโรค',
 'กรมควบคุมโรค กระทรวงสาธารณสุข',
 'กรมควบคุมโรค สถาบันวิจัยระบบสาธารณสุข กระทรวงพาณิชย์ กระทรวงอุตสาหกรรม สำนักงานสถิติแห่งชาติ UNDP',
 'กรมควบคุมโรค)',
 'กรมควมคุมโรค',
 'กรมชลประทาน',
 'กรมพลังงาน เกี่ยวกับพลังงานชุมชน',
 'กรมส่งเสริมคุณภาพสิ่งแวดล้อม',
 'กรมส่งเสริมคุณภาพสิ่งแวดล้อม กรมทรัพยากรทะเลและชายฝั่ง กรมอุทยานแห่งชาติ สัตว์ป่า และพันธุ์พืช',
 'กรมอนามัย',
 'กรมอนามัย กรมควบคุมโรค สถาบันการแพทย์ฉุกเฉินแห่งชาติ  สถาบันพัฒนาองค์กรชุมชน',
 'กรมอนามัย สาธารณสุข',
 'กระทรวงการพัฒนาสังคมและความมั่นคงของมนุษย์',
 'กระทรวงพัฒนาสังคมฯ สถาบันพระปกเกล้า 

In [ ]:
evaFrame_df['กรอบการประเมินผล (Evaluation frameworks) ที่ใช้ประเมินผล'][617]

list

## **Get Latitude & Longitude**

In [ ]:
member_df['ที่อยู่'].to_list()

[nan,
 'เลขที่ 119/36 ซอยท่าอิฐ หมู่บ้านมณียา ถนนรัตนาธิเบศร์ ต.ไทรม้า อ.เมือง จ.นนทบุรี 11000',
 'สาขาพัฒนาสังคม คณะมนุษยศาสตร์และสังคมศาสตร์ มหาวิทยาลัยนเรศวร วิทยาอ.หนองอ้อ อ.เมือง จ.พิษณุโลก 56000',
 'ภาควิชาวิจัยและจิตวิทยาการศึกษา คณะครุศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย',
 'ภาควิชาอนามัยครอบครัว คณะสาธารณสุขศาสตร์ มหาวิยาลัยมหิดล',
 nan,
 'สำนักงานพัฒนานโยบายสุขภาพระหว่างประเทศ อาคารคลังพัสดุ ชั้น 2 กระทรวงสาธารณสุข ซอย 6 ถนนติวานนท์ ตำบลตลาดขวัญ อำเภอเมือง จังหวัดนนทบรี 11000',
 'คณะเภสัชศาสตร์ มหาวิทยาลัยขอนแก่น เลขที่ 123  ถนนมิตรภาพ ต.ในเมือง อ.เมือง จ.ขอนแก่น 40002',
 'เลขที่ 2044/23 ซอยเพชรบุรีตัดใหม่ แขวงห้วยขวาง เขตห้วยขวาง กรุงเทพฯ 10310',
 'คณะครุศาสตร์ มรภ.สวนสุนันทา เลขที่ 1 ถ.อู่ทองนอก แขวงวชิระ เขตดุสิต กทม.10300',
 nan,
 nan,
 '98/16 ม.18 ต.คลองหนึ่ง อ.คลองหลวง จ.ปทุมธานี',
 '83/132 ไอ คอนโด ศาลายา',
 'เลขที่ 188 หมู่ที่ 2 ต.เมืองปัก อ.ปักธงชัย จ.นครราชสีมา 30150',
 nan,
 'สถาบันวิจัยประชากรและสังคม ม.มหิดล',
 '78/153 บ้านฟ้าปิยรมย์ เพส 6 ซอย 17 ถนนลำลูกกา ตำบลบึงคำพร้อ

In [ ]:
#set(member_df['ที่อยู่'])
[w for w in member_df['ที่อยู่'].to_list() if re.search('จ.', w)]

TypeError: ignored

In [ ]:
# importing geopy library
from geopy.geocoders import Nominatim
 
# calling the Nominatim tool
loc = Nominatim(user_agent="GetLoc")
 
# entering the location name
getLoc = loc.geocode("นนทบุรี")
 
# printing address
print(getLoc.address)
 
# printing latitude and longitude
print("Latitude = ", getLoc.latitude, "\n")
print("Longitude = ", getLoc.longitude)

เทศบาลนครนนทบุรี, อำเภอเมืองนนทบุรี, จังหวัดนนทบุรี, 11000, ประเทศไทย
Latitude =  13.8428962 

Longitude =  100.4937886


## **Export Data**

In [ ]:
filepath = Path('/content/drive/MyDrive/ThaiNHF All Work/(ThaiNHF) ข้อมูลพัฒนาเสนอโครงการ สสส/นักประเมิน.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True) 
member_df.to_csv(filepath)

filepath = Path('/content/drive/MyDrive/ThaiNHF All Work/(ThaiNHF) ข้อมูลพัฒนาเสนอโครงการ สสส/ระดับประสบการณ์.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True) 
level_df.to_csv(filepath)

filepath = Path('/content/drive/MyDrive/ThaiNHF All Work/(ThaiNHF) ข้อมูลพัฒนาเสนอโครงการ สสส/กรอบการประเมิน.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True) 
evaFrame_df.to_csv(filepath)

filepath = Path('/content/drive/MyDrive/ThaiNHF All Work/(ThaiNHF) ข้อมูลพัฒนาเสนอโครงการ สสส/รูปแบบการดำเนินการ.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True) 
evaAppo_df.to_csv(filepath)

filepath = Path('/content/drive/MyDrive/ThaiNHF All Work/(ThaiNHF) ข้อมูลพัฒนาเสนอโครงการ สสส/ประเด็นเชี่ยวชาญ.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True) 
issue_df.to_csv(filepath)

filepath = Path('/content/drive/MyDrive/ThaiNHF All Work/(ThaiNHF) ข้อมูลพัฒนาเสนอโครงการ สสส/กลุ่มเป้าหมาย.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True) 
target_df.to_csv(filepath)

"""filepath = Path('/content/drive/MyDrive/ThaiNHF All Work/(ThaiNHF) ข้อมูลพัฒนาเสนอโครงการ สสส/กลุ่มเป้าหมาย.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True) 
target_df.to_csv(filepath)"""

"filepath = Path('/content/drive/MyDrive/ThaiNHF All Work/(ThaiNHF) ข้อมูลพัฒนาเสนอโครงการ สสส/กลุ่มเป้าหมาย.csv')  \nfilepath.parent.mkdir(parents=True, exist_ok=True) \ntarget_df.to_csv(filepath)"

In [ ]:
#d = evaFrame_df.set_index('ID')['กรอบการประเมินผล (Evaluation frameworks) ที่ใช้ประเมินผล'].to_dict()
"""evaFrame_df = evaFrame_df.explode('กรอบการประเมินผล (Evaluation frameworks) ที่ใช้ประเมินผล')
print(set(evaFrame_df['กรอบการประเมินผล (Evaluation frameworks) ที่ใช้ประเมินผล']))
check = evaFrame_df['กรอบการประเมินผล (Evaluation frameworks) ที่ใช้ประเมินผล']
evaFrame_df = evaFrame_df[(check.str.contains(',') == False) & ( check != 'None')]
evaFrame_df = evaFrame_df.dropna().reset_index(drop=True)
"""